# Exploratory Data Analysis
Łukasz Janiak  
08-05-2022  
Data from http://www.planecrashinfo.com/database.htm  

## Import Packages

In [80]:
import pandas as pd
import sqlite3
import plotly.express as px
import plotly.io as pio
import calendar
pio.renderers.default = "notebook_connected"

# Import Data and Preview

In [81]:
con = sqlite3.connect('airplanes.db')
data_import = pd.read_sql_query('SELECT * from airplane_accidents_cleaned', con)

data_import.head()

,index,id,date,location,operator,flight_no,route,ac_type,registration,cn_ln,ground,summary,aboard_total,aboard_passengers,aboard_crew,fatalities_total,fatalities_passengers,fatalities_crew,hours,minutes
0,0,0,1908-09-17,"Fort Myer, Virginia",Military - U.S. Army,None,Demonstration,Wright Flyer III,None,1,0.0,"During a demonstration flight, a U.S. Army fly...",2.0,1.0,1.0,1.0,1.0,0.0,17,18
1,1,1,1909-09-07,"Juvisy-sur-Orge, France",None,None,Air show,Wright Byplane,SC1,None,0.0,Eugene Lefebvre was the first pilot to ever be...,1.0,0.0,1.0,1.0,0.0,0.0,None,None
2,2,2,1912-07-12,"Atlantic City, New Jersey",Military - U.S. Navy,None,Test flight,Dirigible,None,None,0.0,First U.S. dirigible Akron exploded just offsh...,5.0,0.0,5.0,5.0,0.0,5.0,6,30
3,3,3,1913-08-06,"Victoria, British Columbia, Canada",Private,None,None,Curtiss seaplane,None,None,0.0,The first fatal airplane accident in Canada oc...,1.0,0.0,1.0,1.0,0.0,1.0,None,None
4,4,4,1913-09-09,Over the North Sea,Military - German Navy,None,None,Zeppelin L-1 (airship),None,None,0.0,The airship flew into a thunderstorm and encou...,20.0,NaN,NaN,14.0,NaN,NaN,18,30


Columns 'index' and 'id' are redundant so let's drop them.

In [82]:
data_import = data_import.drop(['index', 'id'], axis=1)

In [83]:
data_import.head()

,date,location,operator,flight_no,route,ac_type,registration,cn_ln,ground,summary,aboard_total,aboard_passengers,aboard_crew,fatalities_total,fatalities_passengers,fatalities_crew,hours,minutes
0,1908-09-17,"Fort Myer, Virginia",Military - U.S. Army,None,Demonstration,Wright Flyer III,None,1,0.0,"During a demonstration flight, a U.S. Army fly...",2.0,1.0,1.0,1.0,1.0,0.0,17,18
1,1909-09-07,"Juvisy-sur-Orge, France",None,None,Air show,Wright Byplane,SC1,None,0.0,Eugene Lefebvre was the first pilot to ever be...,1.0,0.0,1.0,1.0,0.0,0.0,None,None
2,1912-07-12,"Atlantic City, New Jersey",Military - U.S. Navy,None,Test flight,Dirigible,None,None,0.0,First U.S. dirigible Akron exploded just offsh...,5.0,0.0,5.0,5.0,0.0,5.0,6,30
3,1913-08-06,"Victoria, British Columbia, Canada",Private,None,None,Curtiss seaplane,None,None,0.0,The first fatal airplane accident in Canada oc...,1.0,0.0,1.0,1.0,0.0,1.0,None,None
4,1913-09-09,Over the North Sea,Military - German Navy,None,None,Zeppelin L-1 (airship),None,None,0.0,The airship flew into a thunderstorm and encou...,20.0,NaN,NaN,14.0,NaN,NaN,18,30


In [84]:
data_import.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5016 entries, 0 to 5015
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   date                   5016 non-null   object 
 1   location               5011 non-null   object 
 2   operator               5006 non-null   object 
 3   flight_no              1333 non-null   object 
 4   route                  4255 non-null   object 
 5   ac_type                5003 non-null   object 
 6   registration           4743 non-null   object 
 7   cn_ln                  4348 non-null   object 
 8   ground                 4972 non-null   float64
 9   summary                4957 non-null   object 
 10  aboard_total           4999 non-null   float64
 11  aboard_passengers      4796 non-null   float64
 12  aboard_crew            4798 non-null   float64
 13  fatalities_total       5008 non-null   float64
 14  fatalities_passengers  4782 non-null   float64
 15  fata

The "date" column is of an object type, not a date type. Let's fix it , rename 'hours' and 'minutes' columns and also add separate columns for year, month and day.

In [85]:
data_import['date'] = pd.to_datetime(data_import['date'], format='%Y-%m-%d')

data_import['year'] = pd.DatetimeIndex(data_import['date']).year

data_import['month_number'] = pd.DatetimeIndex(data_import['date']).month
data_import['month_name'] = data_import['month_number'].apply(lambda x: calendar.month_abbr[x])

data_import.rename(columns = {'hours': 'hour', 'minutes': 'minute'}, inplace=True)

data_import.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5016 entries, 0 to 5015
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   date                   5016 non-null   datetime64[ns]
 1   location               5011 non-null   object        
 2   operator               5006 non-null   object        
 3   flight_no              1333 non-null   object        
 4   route                  4255 non-null   object        
 5   ac_type                5003 non-null   object        
 6   registration           4743 non-null   object        
 7   cn_ln                  4348 non-null   object        
 8   ground                 4972 non-null   float64       
 9   summary                4957 non-null   object        
 10  aboard_total           4999 non-null   float64       
 11  aboard_passengers      4796 non-null   float64       
 12  aboard_crew            4798 non-null   float64       
 13  fat

Data contains some missing values, especially in 'hour' and 'minute' columns. Need to keep that in mind for future analysis.

## Analysis

First, let's create columns that contain the number of survivors.

In [86]:
data_import['survived_total'] = data_import['aboard_total'] - data_import['fatalities_total']
data_import['survived_passengers'] = data_import['aboard_passengers'] - data_import['fatalities_passengers']
data_import['survived_crew'] = data_import['aboard_crew'] - data_import['fatalities_crew']

And also column with total number of casualties.

In [87]:
data_import['dead_total'] = data_import['fatalities_total'] + data_import['ground']

In [88]:
data_import.describe()

,ground,aboard_total,aboard_passengers,aboard_crew,fatalities_total,fatalities_passengers,fatalities_crew,year,month_number,survived_total,survived_passengers,survived_crew,dead_total
count,4972.000000,4999.000000,4796.000000,4798.000000,5008.000000,4782.000000,4782.000000,5016.000000,5016.000000,4999.000000,4777.000000,4779.000000,4972.000000
mean,1.724859,31.116823,26.879274,4.523760,22.298522,18.878503,3.588248,1970.880582,6.692185,8.792759,7.967134,0.931157,24.102172
std,55.452876,45.475030,44.033098,3.759299,35.010685,33.936406,3.180358,24.717496,3.531167,30.620341,29.130906,2.648295,66.589296
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1908.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,7.000000,3.000000,2.000000,4.000000,1.000000,2.000000,1951.000000,4.000000,0.000000,0.000000,0.000000,4.000000
50%,0.000000,16.000000,12.000000,4.000000,11.000000,7.000000,3.000000,1970.000000,7.000000,0.000000,0.000000,0.000000,12.000000
75%,0.000000,35.000000,30.000000,6.000000,25.000000,21.000000,5.000000,1992.000000,10.000000,3.000000,2.000000,1.000000,26.000000
max,2750.000000,644.000000,614.000000,83.000000,583.000000,560.000000,43.000000,2022.000000,12.000000,516.000000,503.000000,81.000000,2842.000000


In column 'ground' (which contains total people killed on the ground) 3rd quartile is equal 0 but max is 2750. Let's take a closer look at this.

In [89]:
data_import[data_import['ground'] == 2750]

,date,location,operator,flight_no,route,ac_type,registration,cn_ln,ground,summary,...,fatalities_crew,hour,minute,year,month_number,month_name,survived_total,survived_passengers,survived_crew,dead_total
4351,2001-09-11,"New York City, New York",American Airlines,11,Boston - Los Angeles,Boeing 767-223ER,N334AA,22332/169,2750.0,The aircraft was hijacked shortly after it lef...,...,11.0,8,47,2001,9,Sep,0.0,0.0,0.0,2842.0
4352,2001-09-11,"New York City, New York",United Air Lines,175,Boston - Los Angeles,Boeing B-767-222,N612UA,21873/41,2750.0,The aircraft was hijacked shortly after it lef...,...,9.0,9,3,2001,9,Sep,0.0,0.0,0.0,2815.0


In [90]:
fig = px.scatter(data_import, x='date', y='dead_total',
    title='Casualties number (including killed on the ground) over the years', labels={'ground': 'number of deaths on the ground'})
fig.update_layout(yaxis_title='number of casualties', bargap=0.1)
fig.show()

In [91]:
fig = px.histogram(data_import[(data_import['ground'] > 0) & (data_import['ground'] != 2750)], x='ground',
    title='Ground fatalities per accident distribution excluding September 11 attacks', labels={'ground': 'number of deaths on the ground'})
fig.update_layout(yaxis_title='number of accidents', bargap=0.1)
fig.show()

In [92]:
data_import.ground.value_counts()

0.0       4722
1.0         63
2.0         34
3.0         21
4.0         16
5.0         12
7.0         10
8.0          9
6.0          6
10.0         6
14.0         5
11.0         4
13.0         4
12.0         4
19.0         4
44.0         4
22.0         4
24.0         3
20.0         3
35.0         2
25.0         2
125.0        2
15.0         2
30.0         2
37.0         2
2750.0       2
32.0         1
45.0         1
39.0         1
18.0         1
225.0        1
49.0         1
9.0          1
78.0         1
40.0         1
50.0         1
31.0         1
113.0        1
71.0         1
33.0         1
23.0         1
87.0         1
17.0         1
29.0         1
16.0         1
36.0         1
58.0         1
63.0         1
53.0         1
85.0         1
Name: ground, dtype: int64

About 5% of accidents have fatalities on the ground and most of them have less than 10 deaths. Both records with 'ground' equals 2750 comes from the same accident "September 11 attacks" which is by far the deadliest one.

In [93]:
fig = px.histogram(data_import, x='year', title='Yearly aviation accidents', nbins=200)
fig.update_layout(yaxis_title='number of accidents', bargap=0.1)
fig.show()

In [94]:
fig = px.histogram(data_import, x='year', title='Aviation accidents over decades',
    range_x=(1900, 2030), nbins=13)
fig.update_layout(yaxis_title='number of accidents', bargap=0.1)
fig.show()

In [95]:
fig = px.histogram(data_import, x='year', y='fatalities_total', nbins=200,
    title='Yearly casualties', labels={'fatalities_total': 'fatalities'})
fig.update_layout(yaxis_title='number of casualties', bargap=0.1)
fig.show()

In [96]:
fig = px.histogram(data_import, x='year', y='fatalities_total', title='Casualties over decades',
    range_x=(1900, 2030), nbins=13)
fig.update_layout(yaxis_title='number of casualties', bargap=0.1)
fig.show()

Accidents and fatalities have dropped by around half in the last decade.

In [97]:
fig = px.histogram(data_import, x='month_name', title='Monthly aviation accidents', labels={'month_name': 'month'},
    category_orders=dict(month_name=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']))
fig.update_layout(yaxis_title='number of accidents', bargap=0.1)
fig.show()

In [98]:
fig = px.histogram(data_import, x='month_name', y='fatalities_total',
    title='Monthly casualties', labels={'month_name': 'month'},
    category_orders=dict(month_name=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']))
fig.update_layout(yaxis_title='number of casualties', bargap=0.1)
fig.show()

Accidents occur most frequently in December and least frequently in April.

In [99]:
fig = px.histogram(data_import, x='hour', title='Hourly aviation accidents',
    category_orders=dict(hour=[i for i in range(24)]))
fig.update_layout(yaxis_title='number of accidents', bargap=0.1)
fig.show()

In [100]:
fig = px.histogram(data_import, x='hour', y='fatalities_total',
    title='Hourly casualties', labels={'fatalities_total': 'fatalities'},
    category_orders=dict(hour=[i for i in range(24)]))
fig.update_layout(yaxis_title='number of casualties', bargap=0.1)
fig.show()

Less accidents happens during night hours. Similar trend is visible in number of casualties.

In [101]:
fig = px.bar(data_import, x='year', y=['fatalities_passengers', 'fatalities_crew'], title='Yearly fatalities distinguishing between passengers and crew')
fig.update_layout(yaxis_title='number of fatalities')
fig.show()

In [102]:
survived = data_import['survived_total'].sum()
total = data_import['aboard_total'].sum()
print(f'Survival rate is {survived / total}')

Survival rate is 0.2825724994053474


In [103]:
fig = px.bar(data_import, x='year', y=['survived_total', 'fatalities_total'], title='Survival rate')
fig.show()

Let's take a closer look at year 1999, where survival rate is way above 50%.

In [104]:
high_survival_rate = data_import[(data_import['year'] == 1999) & (data_import['survived_total'] > 0)]
high_survival_rate[['date', 'aboard_total', 'fatalities_total', 'survived_total', 'summary']]


,date,aboard_total,fatalities_total,survived_total,summary
4195,1999-01-16,4.0,3.0,1.0,"After initiating a go-around, the aircraft vee..."
4201,1999-02-25,31.0,4.0,27.0,"The aircraft touched down briefly, overran the..."
4204,1999-03-19,2.0,1.0,1.0,"On approach, the crew of the cargo plane desce..."
4209,1999-05-08,12.0,7.0,5.0,Crashed into the ocean during a heavy rain. It...
4212,1999-06-01,145.0,11.0,134.0,The plane was on a flight from Dallas/Fort Wor...
4217,1999-06-25,2.0,1.0,1.0,During the takeoff the engine failed and the p...
4218,1999-07-01,5.0,1.0,4.0,The cargo plane was shot down by UNITA rebels.
4220,1999-07-04,18.0,5.0,13.0,The aircraft developed engine trouble shortly ...
4224,1999-07-23,517.0,1.0,516.0,Two minutes after taking off from Haneda Airpo...
4228,1999-08-22,315.0,3.0,312.0,While making a landing in strong winds and rai...


Such a high survival rate in 1999 was due to the many hijackings that took place that year.

## Summary

Key findings:
+ September 9 attacks were by far the most fatal accident (5 times more victims than second most fatal accident)
+ Fewer accidents happen at night (around 50% less)
+ Accidents occur most frequently in December (497) and least frequently in April (349)
+ Accidents and fatalities have dropped by around half in the last decade
+ About 5% of accidents have fatalities on the ground 
+ About 28% of participants in air accidents survive them